# Custom Layers

One of the reasons for the success of deep learning can be found in the wide range of layers that can
be used in a deep network. This allows for a tremendous degree of customization and adaptation. For
instance, scientists have invented layers for images, text, pooling, loops, dynamic programming, even for
computer programs. Sooner or later you will encounter a layer that doesn’t exist yet in Torch, or even
better, you will eventually invent a new layer that works well for your problem at hand. This is when it’s
time to build a custom layer. This section shows you how.

Layers without Parameters
Since this is slightly intricate, we start with a custom layer (aka Module) that doesn’t have any inherent parameters. Our first step is very similar to when we introduced modules previously. The following
CenteredLayer class constructs a layer that subtracts the mean from the input. We build it by inheriting from the Module class and implementing the forward method.



In [1]:
import torch
from torch import nn
from torch.autograd import Variable
import torch.nn.functional as F

In [2]:
class CenteredLayer(nn.Module):
  def __init__(self):
      super().__init__()
  def forward(self, x):
    return x - x.mean()

To see how it works let’s feed some data into the layer.



In [3]:
layer = CenteredLayer()
layer(torch.FloatTensor([1, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

We can also use it to construct more complex models.


In [4]:
net = nn.Sequential(nn.Linear(8,128), CenteredLayer())

Let’s see whether the centering layer did its job. For that we send random data through the network and
check whether the mean vanishes. Note that since we’re dealing with floating point numbers, we’re going
to see a very small albeit typically nonzero number.

In [5]:
y=net(torch.rand(4,8))
y.mean()

tensor(-2.7940e-09, grad_fn=<MeanBackward1>)

# Layers with Parameters

Now that we know how to define layers in principle, let’s define layers with parameters. These can be adjusted through training. In order to simplify things for an avid deep learning researcher the Parameter
class and the ParameterDict dictionary provide some basic housekeeping functionality. In particular,
they govern access, initialization, sharing, saving and loading model parameters. For instance, this way
we don’t need to write custom serialization routines for each new custom layer.
For instance, we can use the member variable params of the ParameterDict type that comes with
the Module class. It is a dictionary that maps string type parameter names to model parameters in the
Parameter type. We can create a Parameter instance from ParameterDict via the get function.

In [6]:
params=torch.nn.ParameterDict()
params.update({"param2":nn.Parameter(Variable(torch.ones(2,3)))})
params

ParameterDict(  (param2): Parameter containing: [torch.FloatTensor of size 2x3])

Let’s use this to implement our own version of the dense layer. It has two parameters - bias and weight.
To make it a bit nonstandard, we bake in the ReLU activation as default. Next, we implement a fully
connected layer with both weight and bias parameters. It uses ReLU as an activation function, where
in_units and units are the number of inputs and the number of outputs, respectively.


In [7]:
class MyDense(nn.Module):
    def __init__(self, units, in_units):
        super().__init__()
        self.weight = Variable(torch.ones(in_units, units))
        self.bias = Variable(torch.ones(units,))
    def forward(self, x):
        linear = torch.matmul(x, self.weight.data) + self.bias.data
        return F.relu(linear)

Naming the parameters allows us to access them by name through dictionary lookup later. It’s a good idea
to give them instructive names. Next, we instantiate the MyDense class and access its model parameters.


In [8]:
dense = MyDense(units=3, in_units=5)
dense.parameters

<bound method Module.parameters of MyDense()>

We can directly carry out forward calculations using custom layers.

In [9]:
dense(torch.rand(2,5))

tensor([[3.1822, 3.1822, 3.1822],
        [2.6808, 2.6808, 2.6808]])

We can also construct models using custom layers. Once we have that we can use it just like the built-in
dense layer. The only exception is that in our case size inference is not automagic. Please consult the
PyTorch documentation for details on how to do this.,

In [10]:
net = nn.Sequential(
MyDense(8, in_units=64),
MyDense(1, in_units=8))

net(torch.rand(2,64))

tensor([[233.8275],
        [246.2336]])

# Summary

• We can design custom layers via the Module class. This is more powerful than defining a module
factory, since it can be invoked in many contexts.

• Modules can have local parameters.

# Exercises

1. Design a layer that learns an affine transform of the data, i.e. it removes the mean and learns an
additive parameter instead.

2. Design a layer that takes an input and computes a tensor reduction, i.e. it returns
∑
yk =
i,j Wijkxixj .

3. Design a layer that returns the leading half of the Fourier coefficients of the data. Hint - look up
the fft function in PyTorch.
